Importo las librerias que son necesarias para Generar el ETL

In [124]:
import pandas as pd
import ast
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_distances
from dateutil import parser
import gzip

### ETL de `output_steam_games`

Leo el JSON output_steam_games

In [3]:
# Importar el Data set
dataset = pd.read_json('output_steam_games.json', lines=True)

Leo el dataframe para ver que corrio bien

In [5]:
dataset

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


Veo la forma del dataset con Shape

In [4]:
dataset.shape

(120445, 13)

Analizo si hay ID repetidas

In [5]:
call_id_counts = dataset['id'].value_counts()
call_id_counts

id
612880.0    2
761140.0    1
530200.0    1
518690.0    1
513460.0    1
           ..
676060.0    1
494160.0    1
215280.0    1
667090.0    1
681550.0    1
Name: count, Length: 32132, dtype: int64

Verifico cuantas se repiten

In [6]:
id_mas_de_una_vez = call_id_counts[call_id_counts > 1].index.tolist()

print(id_mas_de_una_vez)

[612880.0]


Busco las filas que tienen los ID repetidos para su analisis

In [7]:
fila_resultado = dataset[dataset['id'] == 612880.0]
fila_resultado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


Al ver que son el mismo elemento repetido, procedo con la eliminacion de una de ellas

In [8]:
dataset = dataset.drop(102883)

Ya se encuentra sin repetidos el dataset

In [9]:
fila_resultado = dataset[dataset['id'] == 612880.0]
fila_resultado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


Ahora procedo a eliminar los NAN de las primeras filas(obtuve el resultado viendo hasta donde llegaban estas filas vacias en el CSV)

In [10]:
dataset = dataset.drop(range(0, 88384))

In [12]:
dataset

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,0.0,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich


Se eliminan las columnas que no son necesarias

In [13]:
dataset.drop(['title',	'url',	'tags', 'specs', 'early_access', 'reviews_url'], axis=1, inplace=True)

Despues de un analisis de datos desde el dataset, determinamos que si eliminamos los nulos, no hay informacion relevante que se pierda haciendo un `dropna()`

In [14]:
dataset = dataset.dropna()

In [15]:
dataset

,publisher,genres,app_name,release_date,price,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018-01-04,1.99,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660.0,Laush Dmitriy Sergeevich


Cambiamos el nombre de la columna `id` renombrandola `item_id` para futuros merges

In [16]:
dataset = dataset.rename(columns={'id': 'item_id'})

Convierte todos los valores de tipo cadena a 0 en la columna `price`

In [17]:
columna_a_convertir = 'price'

dataset.loc[dataset[columna_a_convertir].apply(lambda x: isinstance(x, str)), columna_a_convertir] = 0

Genero un Parquet desde el Dataframe

In [18]:
dataset.to_parquet('output.parquet', index=False)

Genero un CSV desde el Dataframe

In [20]:
dataset.to_csv('output.csv', index=False)

### ETL de `australian_user_items`

Leo el JSON australian_user_items

In [125]:
data_list = []

In [127]:
#Ruta del archivo JSON
file_path = 'australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list)

In [24]:
data_it

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Se desanidan los datos de la columna ['items']

In [128]:
data_it1 = data_it.explode(['items'])
data_it2 = pd.json_normalize(data_it1['items']).set_index(data_it1['items'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

In [26]:
data_it3

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url,items
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


Se eliminan las columnas que no se consideran relevantes para el posterior analisis

In [129]:
data_it3.drop(['items_count','user_url','item_name','steam_id'], axis=1, inplace=True)

Se eliminan los datos nulos por columnas

In [130]:
data_it3 = data_it3.dropna(subset=['item_id'])
data_it3 = data_it3.dropna(subset=['playtime_forever'])

Se eliminan 2 columnas adicionales

In [131]:
data_it3.drop(['items','playtime_2weeks'], axis=1, inplace=True)

Se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)

In [132]:
data_it3 = data_it3[data_it3['playtime_forever'] != 0]

Se conservan las 3 columnas para el analisis

In [133]:
data_it3 = data_it3[['user_id','item_id','playtime_forever']]

Creo el archivo en formato parquet para reduccion de peso

In [134]:
data_it3.to_parquet('user_items.parquet', index= False)

Se exporta la tabla como archivo separa por comas de nombre 'user_items'

In [135]:
data_it3.to_csv('user_items.csv', index=False)

### ETL de `australian_user_reviews`

Leo el JSON australian_user_reviews

In [89]:
data_list1 = []

In [136]:
#Ruta del archivo JSON
file_path = 'australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list1.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list1)

In [91]:
data_it

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se desanidan los datos de la columna ['reviews']


In [137]:
data_it1 = data_it.explode(['reviews'])
data_it2 = pd.json_normalize(data_it1['reviews']).set_index(data_it1['reviews'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

In [38]:
data_it3

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...,...,...,...,...
25797,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


Se eliminan las columnas que no se consideran relevantes para el posterior analisis


In [138]:
data_it3.drop(['funny','last_edited','helpful','user_url'], axis=1, inplace=True)

In [40]:
data_it3

,posted,item_id,recommend,review,user_id,reviews
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,Posted July 20.,730,True,:D,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


Se eliminan los datos nulos por columnas

In [139]:
#se eliminan los datos nulos por columnas
data_it3 = data_it3.dropna(subset=['posted'])
data_it3 = data_it3.dropna(subset=['item_id'])

In [42]:
data_it3

,posted,item_id,recommend,review,user_id,reviews
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,Posted July 20.,730,True,:D,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


Se eliminan 2 columnas adicionales

In [140]:
data_it3.drop(['reviews'], axis=1, inplace=True)

Se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)

In [141]:
#se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)
data_it3 = data_it3[data_it3['item_id'] != 0]


In [79]:
data_it3

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
25798,Posted July 20.,730,True,:D,LydiaMorley


Se eliminan los duplicados y valores nulos en caso de haber

In [142]:
data_it3 = data_it3.drop_duplicates()
data_it3 = data_it3.dropna()

Se cambia el formato de fecha de posted

In [143]:
def convertir_a_fecha(fecha_str):
    try:
        fecha_str = fecha_str.replace('Posted ', '')

        # Intentar analizar la fecha
        fecha = parser.parse(fecha_str, fuzzy=True)
        
        # Si no se especifica el año, establecerlo como "ND"
        if fecha.year == parser.DEFAULTPARSER:
            return "ND"
        
        return fecha
    except:
        return None

In [144]:
data_it3['posted'] = data_it3['posted'].apply(convertir_a_fecha)

In [145]:
data_it3

,posted,item_id,recommend,review,user_id
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
0,2011-07-15,22200,True,It's unique and worth a playthrough.,76561197970982479
0,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
1,2014-06-24,251610,True,I know what you think when you see this title ...,js41637
1,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
25797,2024-07-10,70,True,a must have classic from steam definitely wort...,76561198312638244
25797,2024-07-08,362890,True,this game is a perfect remake of the original ...,76561198312638244
25798,2024-07-03,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
25798,2024-07-20,730,True,:D,LydiaMorley


Se guarda como formato Parquet y CSV para el merge

In [101]:
data_it3.to_parquet('user_reviews.parquet', index=False)

In [146]:
data_it3.to_csv('user_reviews.csv', index=False)

Se crea el analisis de sentimiento sobre la columna review

In [147]:
from textblob import TextBlob

In [148]:
def analyze_sentiment(review):
    analysis = TextBlob(str(review))
    # Asigna un valor según el análisis de sentimiento
    if analysis.sentiment.polarity > 0.1: 
        return 2  # Positivo
    elif analysis.sentiment.polarity < -0.1:
        return 0  # Malo
    else:
        return 1  # Neutral

Se agrega la columna Sentiment_analysis

In [149]:
data_it3['Sentiment_analysis'] = data_it3['review'].apply(analyze_sentiment)

In [150]:
data_it3

,posted,item_id,recommend,review,user_id,Sentiment_analysis
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
0,2011-07-15,22200,True,It's unique and worth a playthrough.,76561197970982479,2
0,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,1
1,2014-06-24,251610,True,I know what you think when you see this title ...,js41637,2
1,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,js41637,1
...,...,...,...,...,...,...
25797,2024-07-10,70,True,a must have classic from steam definitely wort...,76561198312638244,2
25797,2024-07-08,362890,True,this game is a perfect remake of the original ...,76561198312638244,2
25798,2024-07-03,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,1
25798,2024-07-20,730,True,:D,LydiaMorley,2


Se guarda con el sentiment analisis

In [151]:
data_it3.to_parquet('user_reviews.parquet', index=False)

In [152]:
data_it3.to_csv('user_reviews.csv', index=False)

Volvemos a llamar a los CSV para la creacion de una tabla unica con un JOIN

In [153]:
dfgames = pd.read_csv('output.csv')
dfitems = pd.read_csv('user_items.csv')
dfreviews = pd.read_csv('user_reviews.csv')

Volvemos a chequear si hay datos faltantes

In [154]:
dfgames = dfgames.dropna()

In [155]:
dfitems = dfitems.dropna()

In [156]:
dfreviews = dfreviews.dropna()

Uni los Datasets de Users

In [157]:
df_merge = pd.merge(dfitems,dfreviews, how="left")

In [158]:
df_merge = df_merge.dropna()
df_merge

,user_id,item_id,playtime_forever,posted,recommend,review,Sentiment_analysis
27,76561197970982479,22200,271.0,2011-07-15,True,It's unique and worth a playthrough.,2.0
29,76561197970982479,1250,10006.0,2011-11-05,True,Simple yet with great replayability. In my opi...,2.0
59,76561197970982479,43110,834.0,2011-04-21,True,Great atmosphere. The gunplay can be a bit chu...,1.0
331,js41637,227300,551.0,2013-09-08,True,For a simple (it's actually not all that simpl...,1.0
387,js41637,239030,349.0,2013-11-29,True,Very fun little game to play when your bored o...,1.0
...,...,...,...,...,...,...,...
3285183,Ghoustik,730,3969.0,2024-06-17,True,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...,1.0
3285197,76561198312638244,70,1010.0,2024-07-10,True,a must have classic from steam definitely wort...,2.0
3285198,76561198312638244,130,221.0,2024-07-10,True,if you liked Half life i would really recommen...,2.0
3285210,76561198312638244,233270,533.0,2024-07-21,True,this is a very fun and nice 80s themed shooter...,2.0


Renombro la columna id de output por item_id para que se pueda relacionar

In [159]:
dfgames.rename(columns={'id': 'item_id'}, inplace=True)

In [117]:
dfgames

,publisher,genres,app_name,release_date,price,item_id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...
22524,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018-01-04,1.99,745400.0,Bidoniera Games
22525,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640.0,"Nikita ""Ghost_RUS"""
22526,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530.0,Sacada
22527,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660.0,Laush Dmitriy Sergeevich


Se genera el merge de las 3 tablas

In [160]:
df_merge = pd.merge(dfgames,df_merge, how="left")

In [161]:
df_merge = df_merge.dropna()

In [162]:
df_merge

,publisher,genres,app_name,release_date,price,item_id,developer,user_id,playtime_forever,posted,recommend,review,Sentiment_analysis
18,Stainless Games Ltd,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,1997-06-30,9.99,282010.0,Stainless Games Ltd,InstigatorAU,466.0,2024-01-12,True,"Oh, where do i start? DOS...",1.0
19,Valve,['Action'],Half-Life,1998-11-08,9.99,70.0,Valve,EizanAratoFujimaki,1395.0,2015-10-28,True,ADVERTENCIA:Este Análisis contiene posibles Sp...,1.0
20,Valve,['Action'],Half-Life,1998-11-08,9.99,70.0,Valve,GamerFag,590.0,2011-01-27,True,If you own any half life but not this one that...,1.0
21,Valve,['Action'],Half-Life,1998-11-08,9.99,70.0,Valve,76561198020928326,5599.0,2014-07-01,True,this game has had me hooked for over 4 years a...,1.0
22,Valve,['Action'],Half-Life,1998-11-08,9.99,70.0,Valve,Bluegills,64.0,2013-12-06,True,"This is, by far my most favourite game I have ...",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,76561198015050660,2416.0,2014-12-23,False,作為第一個在steam入手的遊戲，老實說當初是為了1.6來順便帶回來，由於當時網路不太穩的關...,1.0
58176,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,76561198023508728,15.0,2014-10-26,False,i caused 9/11,1.0
58177,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,green290,442.0,2015-01-05,True,This game was suprisingly my first FPS game.11...,2.0
58178,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,174gamecuman700kngkakak,72.0,2014-10-31,True,:v,1.0


Se crea el CSV y se ubica en la carpeta data asi los proximos CSV se alojen en el mismo lugar

In [163]:
df_merge.to_parquet('df_merge.parquet', index=False)

In [164]:
df_merge = pd.read_parquet('df_merge.parquet')

In [165]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38218 entries, 0 to 38217
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   publisher           38218 non-null  object 
 1   genres              38218 non-null  object 
 2   app_name            38218 non-null  object 
 3   release_date        38218 non-null  object 
 4   price               38218 non-null  float64
 5   item_id             38218 non-null  float64
 6   developer           38218 non-null  object 
 7   user_id             38218 non-null  object 
 8   playtime_forever    38218 non-null  float64
 9   posted              38218 non-null  object 
 10  recommend           38218 non-null  bool   
 11  review              38218 non-null  object 
 12  Sentiment_analysis  38218 non-null  float64
dtypes: bool(1), float64(4), object(8)
memory usage: 3.5+ MB


Se cambia el tipo de dato de Free to play a 0 para poder hacer el analisis

In [166]:
df_merge['price'] = df_merge['price'].replace('Free to Play', 0)

Convertir la columna 'price' a tipo numérico y reemplazar NaN con ceros

In [167]:
# Convertir la columna 'price' a tipo numérico y reemplazar NaN con ceros
df_merge['price'] = pd.to_numeric(df_merge['price'], errors='coerce').fillna(0)

# Imprimir los valores no numéricos en la columna 'price' después de la conversión
print("Valores no numéricos en la columna 'price' después de la conversión:")
print(df_merge.loc[df_merge['price'].isnull(),'price'])

Valores no numéricos en la columna 'price' después de la conversión:
Series([], Name: price, dtype: float64)


Se configura la columna release_date a un valor de años

In [168]:
# Supongamos que 'release_date' es la columna que contiene las fechas
df_merge['release_date'] = pd.to_datetime(df_merge['release_date'], errors='coerce')

# Asegúrate de que no haya valores nulos en la columna 'release_date'
print(df_merge['release_date'].isnull().sum())

# Crea la columna 'Años' extrayendo el año después de la conversión
df_merge['Años'] = df_merge['release_date'].dt.year

# Imprime el DataFrame para verificar los resultados
print(df_merge)

186
                 publisher                         genres  \
0      Stainless Games Ltd  ['Action', 'Indie', 'Racing']   
1                    Valve                     ['Action']   
2                    Valve                     ['Action']   
3                    Valve                     ['Action']   
4                    Valve                     ['Action']   
...                    ...                            ...   
38213                Valve                     ['Action']   
38214                Valve                     ['Action']   
38215                Valve                     ['Action']   
38216                Valve                     ['Action']   
38217                Valve                     ['Action']   

                             app_name release_date  price   item_id  \
0                Carmageddon Max Pack   1997-06-30   9.99  282010.0   
1                           Half-Life   1998-11-08   9.99      70.0   
2                           Half-Life   1998-11-08

Configuramos el modelo con 4000 registros para poder manipularlo por cuestiones de peso en render y github

In [169]:
modelo = df_merge.head(4000)

Creamos el Dataset del modelo de entrenamiento

In [11]:
modelo.to_parquet('modelo.parquet', index=False)

Se guarda el Parquet final luego de todo el Preprocesamiento de los datos

In [170]:
df_merge.to_parquet('df_merge.parquet', index=False)

### Hacemos el entrenamiento del modelo de recomendacion

In [ ]:
def entrenar_modelo():
    # Cargar los datos desde los CSVs
    user_items = pd.read_csv('user_items.csv')
    user_reviews = pd.read_csv('user_reviews.csv')
    output = pd.read_csv('output.csv')
    
    # Unir las tablas user_items y user_reviews basadas en la columna 'user_id'
    merged_data = pd.merge(user_items, user_reviews, how="inner")
    #df= merged_data.drop_duplicates()
    df = merged_data.sample(n=5000, random_state=42).drop_duplicates()
   
    promedio_playtime = df['playtime_forever'].mean()
   
    distancia_chica = promedio_playtime / 3
    distancia_media = promedio_playtime - (distancia_chica * 2)

    def asignar_valor(playtime):
        if playtime <= distancia_chica:
            return 0
        elif distancia_chica < playtime <= distancia_media:
            return 1
        else:
            return 2

    # Aplica la función a la columna 'playtime forever' para crear la nueva columna 'nueva_columna'
    df['playtime'] = df['playtime_forever'].apply(asignar_valor)

    user_id_mapping = {user_id: i for i, user_id in enumerate(df['user_id'].unique())}
    df['user_id_numeric'] = df['user_id'].map(user_id_mapping)

    # Mapear item_id a identificadores numéricos únicos
    item_id_mapping = {item_id: i for i, item_id in enumerate(df['item_id'].unique())}
    df['item_id_numeric'] = df['item_id'].map(item_id_mapping)

    # Suma la columna 'playtime_forever' al 'Sentiment_analysis'
    df['combined_rating'] = df['Sentiment_analysis'] + df['playtime']
    
    # Utiliza pivot para crear una matriz de recomendación
    matriz_recomendacion = df.pivot(index='user_id_numeric', columns='item_id_numeric', values='combined_rating')

    # Llenar los valores NaN con 0
    matriz_recomendacion = matriz_recomendacion.fillna(0)

    # Se obtienen los valores de la matriz como un array
    ratings = matriz_recomendacion.values

    # División en conjuntos de entrenamiento y prueba
    ratings_train, _ = train_test_split(ratings, test_size=0.2, random_state=42)

    sim_matrix = 1 - cosine_distances(ratings_train)

    # Número de usuarios y elementos
    num_users, _ = ratings_train.shape

    # Selecciona una parte de la matriz de similitud para el conjunto de entrenamiento
    sim_matrix_train = sim_matrix[:num_users, :num_users]

    datos_entrenamiento = (sim_matrix_train, ratings_train, user_id_mapping, item_id_mapping, df, output)
    
    # Guarda los datos de entrenamiento en un archivo usando pickle
    with gzip.open('datos_entrenamiento.pkl.gz', 'wb') as f:
        pickle.dump(datos_entrenamiento, f)
    
    model = sim_matrix_train, ratings_train, user_id_mapping, item_id_mapping, df, output
    
    return model
entrenar_modelo()